In [24]:
#Importing all libraries
import requests
from bs4 import BeautifulSoup
import csv
import collections
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
#From Github; how to replace and expand contactions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [26]:
#Headers will make it look like you are using a web browser
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
#We will use the iteration to retrieve and scrape the web pages, reviews, and ratings from each page on Yelp
reviews = []
ratings = [] 

neg_reviews = []
pos_reviews = []

review_corpus = []

df = pd.read_csv('ARBY LOW DENSITY REVIEW.csv')

for url in df['url']:
    url = url
    if 'yelp' in url:
        response = requests.get(url, headers=headers, verify=False).text
        soup = BeautifulSoup(response, "lxml")
    #Looping through 'div' 'review-content' will help find all the review containers we need in each page that have rating and review
        for s in soup.find_all("div", attrs={'class': 'review-content'}):
            re = s.find('p', attrs={'lang': 'en'})
    #Makes all the letters lower in reviews
            review = re.text.lower()
            review_corpus.append([review])
    #expandContractions will put the dictionary made earlier to replace the contractions in the reviews
    #Make sure to to run the cList dict cell or else there will be an error
            expandContractions(review)
    #Cleaning the lemmas or words in reviews now will make it easier when we start predictive modeling
            words = word_tokenize(review)
            words = word_tokenize(review.replace('\n',' '))
            clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
            characters_to_remove = ["''",'``','...']
            clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
            english_stops = set(stopwords.words('english'))
            clean_words = [word for word in clean_words if word not in english_stops]
            wordnet_lemmatizer = WordNetLemmatizer()
            lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
            reviews.append(lemma_list)
    #Here we are using a simple control flow to recode the ratings for our model. If rating is greater than 3 positive, else negative   
            rating = s.find_all('img', attrs={'class': 'offscreen'})
    #the rating is actually an image, so we need to convert it into a string and then to an integer
            rate = str(rating)
            int_rating = int(rate[11:12])

            if int_rating == 1 or int_rating == 2 or int_rating == 3:
                rating = 'neg'
                ratings.append('neg')
                neg_reviews.append(lemma_list)
            elif int_rating == 4 or int_rating == 5:
                rating = 'pos'
                ratings.append('pos')
                pos_reviews.append(lemma_list)

my_df = pd.DataFrame(review_corpus)

my_df.to_csv('low_density_arbys_review_corpus.csv', index=False, header=False)

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

In [27]:
data_words_pos = []

data_words_neg = []

for wd_list in pos_reviews:
    for word in wd_list:
        data_words_pos.append(word)
        
for wd_list in neg_reviews:
    for word in wd_list:
        data_words_neg.append(word)

In [28]:
data_words_pos = [data_words_pos]

data_words_neg = [data_words_neg]

In [29]:
# Build the bigram and trigram models
bigram_pos = gensim.models.Phrases(data_words_pos, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_pos = gensim.models.Phrases(bigram_pos[data_words_pos], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_pos = gensim.models.phrases.Phraser(bigram_pos)
trigram_mod_pos = gensim.models.phrases.Phraser(trigram_pos)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [30]:
bigram_neg = gensim.models.Phrases(data_words_neg, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_neg = gensim.models.Phrases(bigram_neg[data_words_neg], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_neg = gensim.models.phrases.Phraser(bigram_neg)
trigram_mod_neg = gensim.models.phrases.Phraser(trigram_neg)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [31]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams_pos(texts):
    return [bigram_mod_pos[doc] for doc in texts]

def make_trigrams_pos(texts):
    return [trigram_mod_pos[bigram_mod_pos[doc]] for doc in texts]

def make_bigrams_neg(texts):
    return [bigram_mod_neg[doc] for doc in texts]

def make_trigrams_neg(texts):
    return [trigram_mod_neg[bigram_mod_neg[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [32]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [33]:
# Remove Stop Words
data_words_pos_nostops = remove_stopwords(data_words_pos)

data_words_neg_nostops = remove_stopwords(data_words_neg)

# Form Bigrams
data_words_bigrams_pos = make_bigrams_pos(data_words_pos_nostops)

data_words_bigrams_neg = make_bigrams_neg(data_words_neg_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_pos = lemmatization(data_words_bigrams_pos, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized_neg = lemmatization(data_words_bigrams_neg, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [34]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized_pos)

# Create Corpus
texts = data_lemmatized_pos

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

id2word_neg = corpora.Dictionary(data_lemmatized_neg)

# Create Corpus
texts = data_lemmatized_neg

# Term Document Frequency
corpus_neg = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [35]:
import pprint

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word_neg,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\n\n\n')
pprint.pprint(lda_model_neg.print_topics())
doc_lda_neg = lda_model_neg[corpus]

[(0,
  '0.002*"food" + 0.002*"arby" + 0.002*"good" + 0.002*"fast" + '
  '0.002*"sandwich" + 0.002*"location" + 0.002*"great" + 0.002*"fresh" + '
  '0.002*"place" + 0.002*"clean"'),
 (1,
  '0.002*"arby" + 0.002*"food" + 0.002*"good" + 0.002*"fast" + '
  '0.002*"sandwich" + 0.002*"location" + 0.002*"place" + 0.002*"clean" + '
  '0.002*"get" + 0.002*"always"'),
 (2,
  '0.002*"arby" + 0.002*"good" + 0.002*"food" + 0.002*"fast" + '
  '0.002*"sandwich" + 0.002*"friendly" + 0.002*"clean" + 0.002*"really" + '
  '0.002*"fry" + 0.002*"great"'),
 (3,
  '0.002*"food" + 0.002*"arby" + 0.002*"fast" + 0.002*"good" + '
  '0.002*"sandwich" + 0.002*"location" + 0.002*"clean" + 0.002*"order" + '
  '0.002*"service" + 0.002*"friendly"'),
 (4,
  '0.002*"arby" + 0.002*"food" + 0.002*"fast" + 0.002*"good" + '
  '0.002*"sandwich" + 0.002*"service" + 0.002*"place" + 0.002*"clean" + '
  '0.002*"go" + 0.002*"fry"'),
 (5,
  '0.002*"food" + 0.002*"arby" + 0.002*"good" + 0.002*"fast" + 0.002*"service" '
  '+ 0.002*"

In [36]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
8     -0.075302 -0.0       1        1  99.988853
4      0.010389 -0.0       2        1   0.001325
5      0.010230  0.0       3        1   0.001316
0      0.009710  0.0       4        1   0.001287
7      0.009050 -0.0       5        1   0.001254
9      0.008138  0.0       6        1   0.001218
1      0.008048  0.0       7        1   0.001217
3      0.007458  0.0       8        1   0.001197
2      0.006474 -0.0       9        1   0.001166
6      0.005805  0.0      10        1   0.001161, topic_info=     Category       Freq      Term      Total  loglift  logprob
term                                                           
28    Default  34.000000      arby  34.000000  30.0000  30.0000
184   Default  33.000000      food  33.000000  29.0000  29.0000
205   Default  25.000000      good  25.000000  28.0000  28.0000
173   Default  24.000000      fast  24.000000  27.0000  27.0000
414   Default  20.000000  sandwich  20.000000  26.0000  26.0000
272   Default  13.000000  location  13.000000  25.0000  25.0000
430   Default  13.000000   service  13.000000  24.0000  24.0000
86    Default  12.000000     clean  12.000000  23.0000  23.0000
195   Default  12.000000       fry  12.000000  22.0000  22.0000
193   Default  11.000000  friendly  11.000000  21.0000  21.0000
352   Default  11.000000     place  11.000000  20.0000  20.0000
328   Default  11.000000     order  11.000000  19.0000  19.0000
200   Default  11.000000       get  11.000000  18.0000  18.0000
17    Default  10.000000    always  10.000000  17.0000  17.0000
417   Default  10.000000     sauce  10.000000  16.0000  16.0000
209   Default  10.000000     great  10.000000  15.0000  15.0000
191   Default  10.000000     fresh  10.000000  14.0000  14.0000
376   Default  10.000000    really  10.000000  13.0000  13.0000
204   Default   9.000000        go   9.000000  12.0000  12.0000
315   Default   9.000000      nice   9.000000  11.0000  11.0000
282   Default   7.000000      make   7.000000  10.0000  10.0000
46    Default   7.000000      beef   7.000000   9.0000   9.0000
326   Default   7.000000     onion   7.000000   8.0000   8.0000
407   Default   6.000000     roast   6.000000   7.0000   7.0000
508   Default   6.000000      time   6.000000   6.0000   6.0000
202   Default   6.000000      give   6.000000   5.0000   5.0000
275   Default   5.000000      look   5.000000   4.0000   4.0000
119   Default   5.000000  customer   5.000000   3.0000   3.0000
217   Default   5.000000      gyro   5.000000   2.0000   2.0000
144   Default   5.000000       eat   5.000000   1.0000   1.0000
...       ...        ...       ...        ...      ...      ...
205   Topic10   0.000034      good  25.916569  -2.1810  -6.0594
414   Topic10   0.000033  sandwich  20.177240  -1.9664  -6.0951
173   Topic10   0.000032      fast  24.012701  -2.1698  -6.1245
430   Topic10   0.000030   service  13.490581  -1.6386  -6.1699
272   Topic10   0.000030  location  13.485936  -1.6540  -6.1856
193   Topic10   0.000029  friendly  11.572506  -1.5208  -6.2054
209   Topic10   0.000029     great  10.616862  -1.4470  -6.2178
417   Topic10   0.000029     sauce  10.616638  -1.4520  -6.2228
376   Topic10   0.000029    really  10.619836  -1.4536  -6.2242
17    Topic10   0.000029    always  10.614190  -1.4546  -6.2257
195   Topic10   0.000029       fry  12.537952  -1.6142  -6.2187
200   Topic10   0.000029       get  11.575780  -1.5398  -6.2242
191   Topic10   0.000029     fresh  10.617338  -1.4582  -6.2290
352   Topic10   0.000029     place  11.572472  -1.5409  -6.2256
86    Topic10   0.000029     clean  12.529912  -1.6223  -6.2275
315   Topic10   0.000028      nice   9.663203  -1.3817  -6.2467
144   Topic10   0.000028       eat   5.835998  -0.9004  -6.2697
328   Topic10   0.000028     order  11.575496  -1.5581  -6.2425
407   Topic10   0.000028     roast   6.791908  -1.0495  -6.2670
204   Topic10   0.000028        go   9.66

In [37]:
vis_neg = pyLDAvis.gensim.prepare(lda_model_neg, corpus_neg, id2word_neg)
vis_neg

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
4     -0.162982 -0.0       1        1  99.991432
0      0.020657 -0.0       2        1   0.001051
3      0.020549 -0.0       3        1   0.001041
9      0.019200  0.0       4        1   0.000979
8      0.018362  0.0       5        1   0.000949
7      0.018270  0.0       6        1   0.000945
5      0.017268 -0.0       7        1   0.000919
1      0.016933 -0.0       8        1   0.000911
6      0.016709 -0.0       9        1   0.000906
2      0.015033  0.0      10        1   0.000876, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
28    Default  43.000000       anywhere  43.000000  30.0000  30.0000
184   Default  42.000000        counter  42.000000  29.0000  29.0000
205   Default  32.000000       daughter  32.000000  28.0000  28.0000
173   Default  30.000000    continually  30.000000  27.0000  27.0000
414   Default  25.000000           iced  25.000000  26.0000  26.0000
272   Default  17.000000         entire  17.000000  25.0000  25.0000
430   Default  17.000000         insult  17.000000  24.0000  24.0000
195   Default  15.000000        crunchy  15.000000  23.0000  23.0000
86    Default  15.000000          bread  15.000000  22.0000  22.0000
200   Default  14.000000       customer  14.000000  21.0000  21.0000
193   Default  14.000000          crowd  14.000000  20.0000  20.0000
328   Default  14.000000            fly  14.000000  19.0000  19.0000
352   Default  14.000000     girlfriend  14.000000  18.0000  18.0000
191   Default  13.000000          creep  13.000000  17.0000  17.0000
209   Default  13.000000          death  13.000000  16.0000  16.0000
417   Default  13.000000            ill  13.000000  15.0000  15.0000
376   Default  13.000000           half  13.000000  14.0000  14.0000
17    Default  13.000000          allow  13.000000  13.0000  13.0000
315   Default  12.000000          filth  12.000000  12.0000  12.0000
204   Default  12.000000          dance  12.000000  11.0000  11.0000
46    Default   9.000000  automatically   9.000000  10.0000  10.0000
326   Default   9.000000           flip   9.000000   9.0000   9.0000
282   Default   9.000000        exactly   9.000000   8.0000   8.0000
508   Default   8.000000           meat   8.000000   7.0000   7.0000
202   Default   8.000000            dab   8.000000   6.0000   6.0000
407   Default   8.000000        however   8.000000   5.0000   5.0000
144   Default   7.000000          close   7.000000   4.0000   4.0000
275   Default   7.000000     especially   7.000000   3.0000   3.0000
15    Default   7.000000          ahead   7.000000   2.0000   2.0000
157   Default   7.000000      complaint   7.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
184   Topic10   0.000021        counter  42.545116  -2.8967  -6.5461
205   Topic10   0.000019       daughter  32.855324  -2.6928  -6.6007
684   Topic10   0.000015        refresh   0.122162   2.6153  -6.8871
414   Topic10   0.000018           iced  25.574995  -2.5002  -6.6586
191   Topic10   0.000018          creep  13.448119  -1.9055  -6.7067
376   Topic10   0.000017           half  13.452872  -1.9154  -6.7162
204   Topic10   0.000017          dance  12.243873  -1.8385  -6.7335
430   Topic10   0.000017         insult  17.091059  -2.1683  -6.7297
417   Topic10   0.000017            ill  13.451217  -1.9360  -6.7370
352   Topic10   0.000017     girlfriend  14.663812  -2.0219  -6.7366
200   Topic10   0.000017       customer  14.658982  -2.0252  -6.7402
17    Topic10   0.000017          allow  13.454393  -1.9426  -6.7433
86    Topic10   0.000017          bread  15.873738  -2.1070  -6.7424
195   Topic10   0.000017        crunchy  15.873659  -2.1109  -6.7463
193   Topic10   0.000017          crowd  14.660439  -2.0357  -6.7505
209   Topic10   0.000017          death  13.450936  -1.95